# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [328]:
// TODO: Replace <FILL IN> with appropriate code

val housing = spark.read.format("csv").option("inferSchema", true).option("header", "true").load("data/housing.csv")

housing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 8 more fields]


---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [329]:
// TODO: Replace <FILL IN> with appropriate code

housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [330]:
// TODO: Replace <FILL IN> with appropriate code
//problem

housing.count()

res259: Long = 20640


## 2.2. Look at the data
Print the first five records of the dataset.

In [331]:
// TODO: Replace <FILL IN> with appropriate code

housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Print the number of records with population more than 10000.

In [332]:
// TODO: Replace <FILL IN> with appropriate code
//problem

housing.filter($"population" > 10000).count()

res261: Long = 23


## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [333]:
// TODO: Replace <FILL IN> with appropriate code
//problem

housing.describe("housing_median_age", "total_rooms", "median_house_value", "population").show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [334]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.sql.functions._

housing.select(max("housing_median_age"), min("total_rooms"), avg("median_house_value")).show()

+-----------------------+----------------+-----------------------+
|max(housing_median_age)|min(total_rooms)|avg(median_house_value)|
+-----------------------+----------------+-----------------------+
|                   52.0|             2.0|     206855.81690891474|
+-----------------------+----------------+-----------------------+



import org.apache.spark.sql.functions._


## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [335]:
// TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity").count().sort("count").show()

+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|         ISLAND|    5|
|       NEAR BAY| 2290|
|     NEAR OCEAN| 2658|
|         INLAND| 6551|
|      <1H OCEAN| 9136|
+---------------+-----+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [336]:
// TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity").agg(avg("median_house_value") as "avg_value").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [337]:
// TODO: Replace <FILL IN> with appropriate code

housing.createOrReplaceTempView("df")
val housingSQL = spark.sql("SELECT ocean_proximity, AVG(median_house_value) FROM df GROUP BY ocean_proximity")
housingSQL.show()
housingSQL.withColumnRenamed("avg(median_house_value)", "avg_value").show()

+---------------+-----------------------+
|ocean_proximity|avg(median_house_value)|
+---------------+-----------------------+
|         ISLAND|               380440.0|
|     NEAR OCEAN|     249433.97742663656|
|       NEAR BAY|     259212.31179039303|
|      <1H OCEAN|     240084.28546409807|
|         INLAND|     124805.39200122119|
+---------------+-----------------------+

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



housingSQL: org.apache.spark.sql.DataFrame = [ocean_proximity: string, avg(median_house_value): double]


## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [338]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("housing_median_age", "total_rooms", "median_house_value", "population")).setOutputCol("features")

val housingAttrs = va.transform(housing)

housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|[41.0,880.0,45260...|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|[21.0,7099.0,3585...|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|[52.0,1467.0,3521...|
|  -122.25|   37.85|              52.0|     12

import org.apache.spark.ml.feature.VectorAssembler
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_299e4ed3a1da
housingAttrs: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]


In [339]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs, "features").head

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293   
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772    
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235  
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0                   


import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
coeff: org.apache.spark.ml.linalg.Matrix =
1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [340]:
// TODO: Replace <FILL IN> with appropriate code

val housingCol1 = housing.withColumn("rooms_per_household", col("total_rooms") / col("households"))
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", col("total_bedrooms") / col("total_rooms"))
val housingExtra = housingCol2.withColumn("population_per_household", col("population") / col("households"))

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



housingCol1: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]
housingCol2: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 10 more fields]
housingExtra: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [341]:
// TODO: Replace <FILL IN> with appropriate code

val renamedHousing = housingExtra.withColumnRenamed("median_house_value", "label")
//renamedHousing.select("label").show()

renamedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [342]:
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns: remove the categotical column and the label col
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel: String = label
colCat: String = ocean_proximity
colNum: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)


## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [343]:
// TODO: Replace <FILL IN> with appropriate code

for (c <- colNum) {
    val no_missing = renamedHousing.filter(col(c).isNull).count()
    println(s"Number of missing values in column ${c}: ${no_missing}")
}
//renamedHousing.select("total_bedrooms", "bedrooms_per_room").show(100)

Number of missing values in column longitude: 0
Number of missing values in column latitude: 0
Number of missing values in column housing_median_age: 0
Number of missing values in column total_rooms: 0
Number of missing values in column total_bedrooms: 207
Number of missing values in column population: 0
Number of missing values in column households: 0
Number of missing values in column median_income: 0
Number of missing values in column rooms_per_household: 0
Number of missing values in column bedrooms_per_room: 207
Number of missing values in column population_per_household: 0


As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [344]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median").setInputCols(Array("total_bedrooms", "bedrooms_per_room")).setOutputCols(Array("total_bedrooms", "bedrooms_per_room"))                                
val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing) //total transformer (estimator + transformer)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

/*for (c <- colNum) {
    val no_missing = imputedHousing.filter(col(c).isNull).count()
    println(s"Number of missing values in column ${c}: ${no_missing}")
}*/

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows



import org.apache.spark.ml.feature.Imputer
imputer: org.apache.spark.ml.feature.Imputer = imputer_fcf162fa5bbe
imputedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [345]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("features")
val featuredHousing = va.transform(imputedHousing)
//featuredHousing.select("features").show(300)

//StandardScaler is an estimator

val scaler = new StandardScaler().setInputCol("features").setOutputCol("scaledFeatures").setWithMean(true)
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)//.drop("features")

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|            features|      scaledFeatures|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-1.3278030546902...|
|  -122.22|   37.86|              21.0|     7099.0|        1

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_8810f7168cc9
featuredHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 12 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_b377d1383778
scaledHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 13 more fields]


## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [346]:
// TODO: Replace <FILL IN> with appropriate code

renamedHousing.select(countDistinct("ocean_proximity")).show

+-------------------------------+
|count(DISTINCT ocean_proximity)|
+-------------------------------+
|                              5|
+-------------------------------+



### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [347]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol("ocean_proximity").setOutputCol("ocean_ind")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_ind|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|      3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.238137082601054|0.15579659106916466|       2.109841827768

import org.apache.spark.ml.feature.StringIndexer
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_d1491e2a73b9
idxHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 12 more fields]


Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [348]:
indexer.fit(renamedHousing).labels

res275: Array[String] = Array(<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND)


### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [349]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().setInputCols(Array("ocean_ind")).setOutputCols(Array("ocean_one_hot"))
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------+-------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_ind|ocean_one_hot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------+-------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|      3.0|(4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.2

import org.apache.spark.ml.feature.OneHotEncoderEstimator
encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_31e8d2584e83
ohHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 13 more fields]


---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [350]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(imputer, va, scaler))
val catPipeline = new Pipeline().setStages(Array(indexer, encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))
val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing).drop("features")

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+---------+-------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|      scaledFeatures|ocean_ind|ocean_one_hot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+---------+-------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-1.3278030546902...|      3.0|(4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}
numPipeline: org.apache.spark.ml.Pipeline = pipeline_fe49c011a220
catPipeline: org.apache.spark.ml.Pipeline = pipeline_abfff9cbe71d
pipeline: org.apache.spark.ml.Pipeline = pipeline_c3d0c8c1620c
newHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 14 more fields]


Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [351]:
// TODO: Replace <FILL IN> with appropriate code

val va2 = new VectorAssembler().setInputCols(Array("scaledFeatures", "ocean_one_hot" )).setOutputCol("features")
val dataset = va2.transform(newHousing).select("features", "label")

dataset.show(5)

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-1.3278030546902...|452600.0|
|[-1.3228118684350...|358500.0|
|[-1.3327942409452...|352100.0|
|[-1.3377854272003...|341300.0|
|[-1.3377854272003...|342200.0|
+--------------------+--------+
only showing top 5 rows



va2: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_569f734b7aac
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: double]


---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [352]:
// TODO: Replace <FILL IN> with appropriate code

val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: double]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: double]


## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [353]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression().setFeaturesCol("features").setLabelCol("label").setSolver("auto")
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: ${lrModel.coefficients}, Intercept: ${lrModel.intercept}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")

Coefficients: [-55580.52180578753,-55907.82103714552,13744.423583157613,6282.351349423896,1951.8793293766314,-45326.05930641086,42177.71969058122,77974.19868075338,6757.324924480594,16404.712653294762,449.5437905321251,-148575.345058603,-182679.2075400226,-144101.35600715823,-153595.70782622276], Intercept: 365981.2617804804
RMSE: 68263.32011248115


import org.apache.spark.ml.regression.LinearRegression
lr: org.apache.spark.ml.regression.LinearRegression = linReg_56a57e0571c3
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = linReg_56a57e0571c3
trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@b5c15c4


Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [354]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|121420.16160776583| 68400.0|[-2.3160579331997...|
|133451.60274300282| 70500.0|[-2.3010843744344...|
|198438.36351656783|128900.0|[-2.2960931881793...|
|  141800.931868001| 81300.0|[-2.2960931881793...|
|156311.72433652292| 70500.0|[-2.2960931881793...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 66409.49373584041


import org.apache.spark.ml.evaluation.RegressionEvaluator
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_8e24a5c528b7
rmse: Double = 66409.49373584041


## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [355]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|174196.96783980582| 68400.0|[-2.3160579331997...|
|144371.72934855608| 70500.0|[-2.3010843744344...|
|234878.18055555556|128900.0|[-2.2960931881793...|
|144371.72934855608| 81300.0|[-2.2960931881793...|
|150161.00386100388| 70500.0|[-2.2960931881793...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 65914.89909018994


import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
dt: org.apache.spark.ml.regression.DecisionTreeRegressor = dtr_1d322ed75b5c
dtModel: org.apache.spark.ml.regression.DecisionTreeRegressionModel = DecisionTreeRegressionModel (uid=dtr_1d322ed75b5c) of depth 5 with 63 nodes
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_81c80ed65458
rmse: Double = 65914.89909018994


## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [356]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().setLabelCol("label").setFeaturesCol("features").setNumTrees(10)

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|  166558.606930618| 68400.0|[-2.3160579331997...|
| 162419.9825656572| 70500.0|[-2.3010843744344...|
|208886.32338192867|128900.0|[-2.2960931881793...|
| 162419.9825656572| 81300.0|[-2.2960931881793...|
| 153161.0282288857| 70500.0|[-2.2960931881793...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 64828.15604614633


import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
rf: org.apache.spark.ml.regression.RandomForestRegressor = rfr_f965d148d678
rfModel: org.apache.spark.ml.regression.RandomForestRegressionModel = RandomForestRegressionModel (uid=rfr_f965d148d678) with 10 trees
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_ea6b89271e2a
rmse: Double = 64828.15604614633


## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [357]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().setFeaturesCol("features").setLabelCol("label").setMaxIter(10)
                                                                                       
// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|119521.02751986249| 68400.0|[-2.3160579331997...|
|105413.62413239325| 70500.0|[-2.3010843744344...|
|142569.65126172887|128900.0|[-2.2960931881793...|
|105413.62413239325| 81300.0|[-2.2960931881793...|
| 94642.40992766448| 70500.0|[-2.2960931881793...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 57730.41964013859


import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
gb: org.apache.spark.ml.regression.GBTRegressor = gbtr_6fe0f9e82d6e
gbModel: org.apache.spark.ml.regression.GBTRegressionModel = GBTRegressionModel (uid=gbtr_6fe0f9e82d6e) with 10 trees
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_ddde18e469b7
rmse: Double = 57730.41964013859


---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [358]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val rf = new RandomForestRegressor().setLabelCol("label").setFeaturesCol("features")

val paramGrid = new ParamGridBuilder().addGrid(rf.numTrees, Array(1, 5, 10)).addGrid(rf.maxDepth, Array(5, 10, 15)).build

val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val cv = new CrossValidator().setEstimator(rf).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid)
    .setNumFolds(3)

val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+-----------------+--------+--------------------+
|       prediction|   label|            features|
+-----------------+--------+--------------------+
|         119307.5| 68400.0|[-2.3160579331997...|
|86072.99633699634| 70500.0|[-2.3010843744344...|
|         182330.0|128900.0|[-2.2960931881793...|
|85395.30627705627| 81300.0|[-2.2960931881793...|
|81089.29070929073| 70500.0|[-2.2960931881793...|
+-----------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 50468.12519877729


import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator
rf: org.apache.spark.ml.regression.RandomForestRegressor = rfr_2217207b5eed
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfr_2217207b5eed-maxDepth: 5,
	rfr_2217207b5eed-numTrees: 1
}, {
	rfr_2217207b5eed-maxDepth: 5,
	rfr_2217207b5eed-numTrees: 5
}, {
	rfr_2217207b5eed-maxDepth: 5,
	rfr_2217207b5eed-numTrees: 10
}, {
	rfr_2217207b5eed-maxDepth: 10,
	rfr_2217207b5eed-numTrees: 1
}, {
	rfr_2217207b5eed-maxDepth: 10,
	rfr_2217207b5eed-numTrees: 5
}, {
	rfr_2217207b5eed-maxDepth: 10,
	rfr_2217207b5eed-numTrees: 10
}, {
	rfr_2217207b5eed-maxDepth: 15,
	rfr_2217207b5eed-numTrees: 1
}, {
	rfr_2217207b5eed-maxDepth: 15,
...

---
# 7. Custom transformer
At the end of part two, we added extra columns to the `housing` dataset. Here, we are going to implement a Transformer to do the same task. The Transformer should take the name of two input columns `inputCol1` and `inputCol2`, as well as the name of ouput column `outputCol`. It, then, computes `inputCol1` divided by `inputCol2`, and adds its result as a new column to the dataset. The details of the implemeting a custom Tranfomer is explained [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types). Please read it before before starting to implement it.

First, define the given parameters of the Transformer and implement a method to validate their schemas (`StructType`).

In [359]:
import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.ml.param

trait MyParams extends Params {
    final val inputCol1 = new Param[String](this, "inputCol1", "The first input column")
    final val inputCol2 = new Param[String](this, "inputCol2", "The second input column")
    final val outputCol = new Param[String](this, "outputCol", "The output column")
    
  protected def validateAndTransformSchema(schema: StructType): StructType = {
    // Check that the input type is a double
    val id_1 = schema.fieldIndex($(inputCol1))
    val id_2 = schema.fieldIndex($(inputCol2))
    val field_1 = schema.fields(id_1)
    val field_2 = schema.fields(id_2)
    if (field_1.dataType != DoubleType) {
        throw new Exception(s"Input1 type ${field_1.dataType} did not match input type DoubleType")
    }
    if (field_2.dataType != DoubleType) {
        throw new Exception(s"Input2 type ${field_2.dataType} did not match input type DoubleType")
    }
    // Add the return field
    schema.add(StructField($(outputCol), DoubleType, false))
  }
}

import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.ml.param
defined trait MyParams


In [360]:
import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.ml.param

class MyTransformer(override val uid: String) extends Transformer with MyParams {
    def this() = this(Identifiable.randomUID("configurablewordcount"))
    
    def setInputCol1(value: String): this.type = set(inputCol1, value)
    
    def setInputCol2(value: String): this.type = set(inputCol2, value)
    
    def setOutputCol(value: String): this.type = set(outputCol, value)

    override def copy(extra: ParamMap): MyTransformer = {
        defaultCopy(extra)
    }

    override def transformSchema(schema: StructType): StructType = {
        // Add the return field
        schema.add(StructField($(outputCol), DoubleType, false))
      }
    
    override def transform(dataset: Dataset[_]): DataFrame = {
        dataset.withColumn($(outputCol), col($(inputCol1)) / col($(inputCol2)))
  }
}

import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.ml.param
defined class MyTransformer


Now, an instance of `MyTransformer`, and set the input columns `total_rooms` and `households`, and the output column `rooms_per_household` and run it over the `housing` dataset.

In [361]:
val myTransformer = new MyTransformer().setInputCol1("total_rooms").setInputCol2("households").setOutputCol("rooms_per_household")

val myDataset = myTransformer.transform(housing).select("rooms_per_household").show(5)

+-------------------+
|rooms_per_household|
+-------------------+
|  6.984126984126984|
|  6.238137082601054|
|  8.288135593220339|
| 5.8173515981735155|
|  6.281853281853282|
+-------------------+
only showing top 5 rows



myTransformer: MyTransformer = configurablewordcount_eb562fb3be52
myDataset: Unit = ()


---
# 8. Custom estimator (predictor)
Now, it's time to implement your own linear regression with gradient descent algorithm as a brand new Estimator. The whole code of the Estimator is given to you, and you do not need to implement anything. It is just a sample that shows how to build a custom Estimator.

The gradient descent update for linear regression is:
$$
w_{i+1} = w_{i} - \alpha_{i} \sum\limits_{j=1}^n (w_i^\top x_j - y_j)x_j
$$

where $i$ is the iteration number of the gradient descent algorithm, and $j$ identifies the observation. Here, $w$ represents an array of weights that is the same size as the array of features and provides a weight for each of the features when finally computing the label prediction in the form:

$$
prediction = w^\top \cdot\ x
$$

where $w$ is the final array of weights computed by the gradient descent, $x$ is the array of features of the observation point and $prediction$ is the label we predict should be associated to this observation.

The given `Helper` class implements the helper methods:
* `dot`: implements the dot product of two vectors and the dot product of a vector and a scalar
* `sum`: implements addition of two vectors
* `fill`: creates a vector of predefined size and initialize it with the predefined value

What you need to do is to implement the methods of the Linear Regresstion class `LR`, which are
* `rmsd`: computes the Root Mean Square Error of a given RDD of tuples of (label, prediction) using the formula:
$$
rmse = \sqrt{\frac{\sum\limits_{i=1}^n (label - prediction)^2}{n}}
$$
* `gradientSummand`: computes the following formula:
$$
gs_{ij} = (w_i^\top x_j - y_j)x_j
$$
* `gradient`: computes the following formula:
$$
gradient = \sum\limits_{j=1}^n gs_{ij}
$$

In [362]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

case class Instance(label: Double, features: Vector)

object Helper extends Serializable {
  def dot(v1: Vector, v2: Vector): Double = {
    val m = Matrices.dense(1, v1.size, v1.toArray)
    m.multiply(v2).values(0)
  }

  def dot(v: Vector, s: Double): Vector = {
    val baseArray = v.toArray.map(vi => vi * s)
    Vectors.dense(baseArray)
  }

  def sumVectors(v1: Vector, v2: Vector): Vector = {
    val baseArray = ((v1.toArray) zip (v2.toArray)).map { case (val1, val2) => val1 + val2 }
    Vectors.dense(baseArray)
  }

  def fillVector(size: Int, fillVal: Double): Vector = Vectors.dense(Array.fill[Double](size)(fillVal));
}

import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}
defined class Instance
defined object Helper


In [363]:
class LR() extends Serializable {
  def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
    val regressionMetrics = new RegressionMetrics(labelsAndPreds)
    regressionMetrics.rootMeanSquaredError
  }
  
  def gradientSummand(weights: Vector, lp: Instance): Vector = {
    val mult = (Helper.dot(weights, lp.features) - lp.label)
    val seq = (0 to lp.features.size - 1).map(i => lp.features(i) * mult)
    return Vectors.dense(seq.toArray)
  }
  
  def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
    val n = trainData.count()
    val d = trainData.take(1)(0).features.size
    var w = Helper.fillVector(d, 0)
    val alpha = 1.0
    val errorTrain = Array.fill[Double](numIters)(0.0)

    for (i <- 0 until numIters) {
      val labelsAndPredsTrain = trainData.map(lp => (lp.label, Helper.dot(w, lp.features)))
      errorTrain(i) = calcRMSE(labelsAndPredsTrain)

      val gradient = trainData.map(lp => gradientSummand(w, lp)).reduce((v1, v2) => Helper.sumVectors(v1, v2))
      val alpha_i = alpha / (n * scala.math.sqrt(i + 1))
      val wAux = Helper.dot(gradient, (-1) * alpha_i)
      w = Helper.sumVectors(w, wAux)
    }
    (w, errorTrain)
  }
}

<console>: 17: error: not found: type RDD

In [ ]:
abstract class MyLinearModel[FeaturesType, Model <: MyLinearModel[FeaturesType, Model]]
  extends PredictionModel[FeaturesType, Model] {
}

class MyLinearModelImpl(override val uid: String, val weights: Vector, val trainingError: Array[Double])
    extends MyLinearModel[Vector, MyLinearModelImpl] {

  override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)

  def predict(features: Vector): Double = {
    println("Predicting")
    val prediction = Helper.dot(weights, features)
    prediction
  }
}

In [ ]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearRegression[
    FeaturesType,
    Learner <: MyLinearRegression[FeaturesType, Learner, Model],
    Model <: MyLinearModel[FeaturesType, Model]]
  extends Predictor[FeaturesType, Learner, Model] {
}

class MyLinearRegressionImpl(override val uid: String)
    extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
  def this() = this(Identifiable.randomUID("linReg"))

  override def copy(extra: ParamMap): MyLinearRegressionImpl = defaultCopy(extra)
  
  def train(dataset: Dataset[_]): MyLinearModelImpl = {
    println("Training")

    val numIters = 10

    val instances: RDD[Instance] = dataset.select(
      col($(labelCol)), col($(featuresCol))).rdd.map {
        case Row(label: Double, features: Vector) =>
          Instance(label, features)
      }

    val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)

    new MyLinearModelImpl(uid, weights, trainingError)
  }
}

In [ ]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
val model = lr.fit(trainSet)
val predictions = model.transform(trainSet)
predictions.select("prediction", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

---
# 9. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

## 9.1 Load the Data

In [364]:
val credit = spark.read.format("csv").option("inferSchema", true).option("header", true).load("data/ccdefault.csv")

credit: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 23 more fields]


## 9.2 Show the Data

In [365]:
credit.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- DEFAULT: integer (nullable = tru

In [366]:
credit.count()

res286: Long = 30000


In [367]:
credit.show(10)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|     3261|       0|    1000|    1000|    1000|       0|    2000|    

## 9.3 Exploraty Analysis

In [368]:
for (c <- credit.columns) {
    println(s"Number of distinct values in col $c: ${credit.select(c).distinct().count()}")
}

Number of distinct values in col ID: 30000
Number of distinct values in col LIMIT_BAL: 81
Number of distinct values in col SEX: 2
Number of distinct values in col EDUCATION: 7
Number of distinct values in col MARRIAGE: 4
Number of distinct values in col AGE: 56
Number of distinct values in col PAY_0: 11
Number of distinct values in col PAY_2: 11
Number of distinct values in col PAY_3: 11
Number of distinct values in col PAY_4: 11
Number of distinct values in col PAY_5: 10
Number of distinct values in col PAY_6: 10
Number of distinct values in col BILL_AMT1: 22723
Number of distinct values in col BILL_AMT2: 22346
Number of distinct values in col BILL_AMT3: 22026
Number of distinct values in col BILL_AMT4: 21548
Number of distinct values in col BILL_AMT5: 21010
Number of distinct values in col BILL_AMT6: 20604
Number of distinct values in col PAY_AMT1: 7943
Number of distinct values in col PAY_AMT2: 7899
Number of distinct values in col PAY_AMT3: 7518
Number of distinct values in col PAY

In [369]:
credit.groupBy("DEFAULT").count().show()

+-------+-----+
|DEFAULT|count|
+-------+-----+
|      1| 6636|
|      0|23364|
+-------+-----+



In [370]:
credit.describe("LIMIT_BAL", "AGE", "PAY_0", "BILL_AMT1", "PAY_AMT1").show()

+-------+------------------+-----------------+------------------+-----------------+-----------------+
|summary|         LIMIT_BAL|              AGE|             PAY_0|        BILL_AMT1|         PAY_AMT1|
+-------+------------------+-----------------+------------------+-----------------+-----------------+
|  count|             30000|            30000|             30000|            30000|            30000|
|   mean|167484.32266666667|          35.4855|           -0.0167|       51223.3309|        5663.5805|
| stddev|129747.66156720246|9.217904068090155|1.1238015279973335|73635.86057552966|16563.28035402577|
|    min|             10000|               21|                -2|          -165580|                0|
|    max|           1000000|               79|                 8|           964511|           873552|
+-------+------------------+-----------------+------------------+-----------------+-----------------+



In [371]:
credit.describe("SEX", "EDUCATION", "MARRIAGE").show()

+-------+------------------+------------------+------------------+
|summary|               SEX|         EDUCATION|          MARRIAGE|
+-------+------------------+------------------+------------------+
|  count|             30000|             30000|             30000|
|   mean|1.6037333333333332|1.8531333333333333|1.5518666666666667|
| stddev|0.4891291960902602|0.7903486597207269|0.5219696006132467|
|    min|                 1|                 0|                 0|
|    max|                 2|                 6|                 3|
+-------+------------------+------------------+------------------+



In [372]:
import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("LIMIT_BAL", "AGE", "PAY_0", "BILL_AMT1", "PAY_AMT1")).setOutputCol("attributes")

val creditAttrs = va.transform(credit)

creditAttrs.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+--------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|          attributes|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+--------------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|[20000.0,24.0,2.0...|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|    

import org.apache.spark.ml.feature.VectorAssembler
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_4eb3ea95fc70
creditAttrs: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 24 more fields]


In [373]:
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(creditAttrs, "attributes").head

println(s"The standard correlation coefficient:\n ${coeff.toString(5, Int.MaxValue)}")

The standard correlation coefficient:
 1.0                   0.14471279755736938   -0.27121433213471846  0.28542986496499084  0.19523591523220815   
0.14471279755736938   1.0                   -0.03944737618411289  0.05623853326059214  0.026146679056997     
-0.27121433213471846  -0.03944737618411289  1.0                   0.18706843147380176  -0.07926871118541942  
0.28542986496499084   0.05623853326059214   0.18706843147380176   1.0                  0.14027727757532402   
0.19523591523220815   0.026146679056997     -0.07926871118541942  0.14027727757532402  1.0                   


import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
coeff: org.apache.spark.ml.linalg.Matrix =
1.0                   0.14471279755736938   -0.27121433213471846  ... (5 total)
0.14471279755736938   1.0                   -0.03944737618411289  ...
-0.27121433213471846  -0.03944737618411289  1.0                   ...
0.28542986496499084   0.05623853326059214   0.18706843147380176   ...
0.19523591523220815   0.026146679056997     -0.07926871118541942  ...


## 9.4 Prepare the Data for Machine Learning Algorithms

In [374]:
val renamedCredit = credit.withColumnRenamed("DEFAULT", "label")
//val cleanedCredit = renamedCredit.drop("ID")
renamedCredit.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|    1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|     3261|       0|    1000|    1000|    1000|       0|    2000|    1|
|  3|

renamedCredit: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 23 more fields]


In [375]:
//label column
val colLabel = "label"

//categorical columns
val colCat = Array("SEX", "EDUCATION", "MARRIAGE")
//val colCat = "SEX"

// numerical columns: remove the categotical column and the label col
//val colNum = cleanedCredit.columns.filter(_ != colLabel).filter(!colCat.contains(_))
val colNum = Array("LIMIT_BAL", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4","PAY_AMT5", "PAY_AMT6")
//renamedCredit.show(5)

colLabel: String = label
colCat: Array[String] = Array(SEX, EDUCATION, MARRIAGE)
colNum: Array[String] = Array(LIMIT_BAL, AGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)


### 9.4.1 Prepare Continuous Attributes

In [376]:
for (c <- colNum) {
    val no_missing = renamedCredit.filter(col(c).isNull).count()
    println(s"Number of missing values in column ${c}: ${no_missing}")
}

Number of missing values in column LIMIT_BAL: 0
Number of missing values in column AGE: 0
Number of missing values in column PAY_0: 0
Number of missing values in column PAY_2: 0
Number of missing values in column PAY_3: 0
Number of missing values in column PAY_4: 0
Number of missing values in column PAY_5: 0
Number of missing values in column PAY_6: 0
Number of missing values in column BILL_AMT1: 0
Number of missing values in column BILL_AMT2: 0
Number of missing values in column BILL_AMT3: 0
Number of missing values in column BILL_AMT4: 0
Number of missing values in column BILL_AMT5: 0
Number of missing values in column BILL_AMT6: 0
Number of missing values in column PAY_AMT1: 0
Number of missing values in column PAY_AMT2: 0
Number of missing values in column PAY_AMT3: 0
Number of missing values in column PAY_AMT4: 0
Number of missing values in column PAY_AMT5: 0
Number of missing values in column PAY_AMT6: 0


In [377]:
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

//for (c <- colNum) {
//   println(c)
//}
renamedCredit.show(6)

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("features")
val featuredCredit = va.transform(renamedCredit)

featuredCredit.select("features").show(6,false)

//StandardScaler

val scaler = new StandardScaler().setInputCol("features").setOutputCol("scaledFeatures").setWithMean(true)
val scaledCredit = scaler.fit(featuredCredit).transform(featuredCredit).drop("features")

scaledCredit.select("scaledFeatures").show(6)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|    1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|     3261|       0|    1000|    1000|    1000|       0|    2000|    1|
|  3|

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_6c52bc995f1c
featuredCredit: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 24 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_863450d1c443
scaledCredit: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 24 more fields]


### 9.4.4 Prepare Categorical Attributes

In [378]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val colCat_ind = Array("SEX_IND", "EDUCATION_IND", "MARRIAGE_IND")
val encoder = new OneHotEncoderEstimator().setInputCols(colCat).setOutputCols(colCat_ind)
val ohCredit = encoder.fit(scaledCredit).transform(scaledCredit)

ohCredit.select("scaledFeatures", "SEX_IND", "EDUCATION_IND", "MARRIAGE_IND", "label").show(5)

+--------------------+-------------+-------------+-------------+-----+
|      scaledFeatures|      SEX_IND|EDUCATION_IND| MARRIAGE_IND|label|
+--------------------+-------------+-------------+-------------+-----+
|[-1.1367012005089...|    (2,[],[])|(6,[2],[1.0])|(3,[1],[1.0])|    1|
|[-0.3659744005642...|    (2,[],[])|(6,[2],[1.0])|(3,[2],[1.0])|    1|
|[-0.5971924405476...|    (2,[],[])|(6,[2],[1.0])|(3,[2],[1.0])|    0|
|[-0.9054831605255...|    (2,[],[])|(6,[2],[1.0])|(3,[1],[1.0])|    0|
|[-0.9054831605255...|(2,[1],[1.0])|(6,[2],[1.0])|(3,[1],[1.0])|    0|
+--------------------+-------------+-------------+-------------+-----+
only showing top 5 rows



import org.apache.spark.ml.feature.OneHotEncoderEstimator
colCat_ind: Array[String] = Array(SEX_IND, EDUCATION_IND, MARRIAGE_IND)
encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_afb5429638be
ohCredit: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 27 more fields]


In [379]:
val va_9 = new VectorAssembler().setInputCols(Array("scaledFeatures", "SEX_IND", "EDUCATION_IND", "MARRIAGE_IND")).setOutputCol("features")
val dataset = va_9.transform(ohCredit).select("features", "label")

dataset.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[-1.1367012005089...|    1|
|[-0.3659744005642...|    1|
|[-0.5971924405476...|    0|
|[-0.9054831605255...|    0|
|[-0.9054831605255...|    0|
+--------------------+-----+
only showing top 5 rows



va_9: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_c9433670b859
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: int]


## 9.5 Make a Model

In [380]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))
trainSet.show(5)
testSet.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[-1.2137738805034...|    0|
|[-1.2137738805034...|    1|
|[-1.2137738805034...|    0|
|[-1.2137738805034...|    0|
|[-1.2137738805034...|    0|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[-1.2137738805034...|    1|
|[-1.2137738805034...|    0|
|[-1.2137738805034...|    0|
|[-1.2137738805034...|    0|
|[-1.2137738805034...|    0|
+--------------------+-----+
only showing top 5 rows



trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]


### 9.5.1 Logistic Regression Model

In [381]:
import org.apache.spark.ml.classification.{BinaryLogisticRegressionSummary, LogisticRegression}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

//Build the logistic Regression Model
val lr = new LogisticRegression().setFeaturesCol("features").setLabelCol("label")//.setMaxIter(50)//.setRegParam(0.3).setElasticNetParam(0.8)

val t1 = System.nanoTime
//Train the logistic Regression Model on trainSet
val lrModel = lr.fit(trainSet)

//Predict on testSet
val predictions = lrModel.transform(testSet)

val duration = (System.nanoTime - t1) / 1e9d

predictions.select("prediction", "label", "features").show(5)

//Judge Model's Performance on test data
//val trainingSummary = lrModel.summary
//val binarySummary = trainingSummary.
val evaluator = new BinaryClassificationEvaluator().setMetricName("areaUnderROC").setLabelCol("label")//.setRawPredictionCol("prediction")
val roc1 = evaluator.evaluate(predictions)
println(s"Area under ROC on test data = $roc1")
println(s"Execution time = $duration")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    1|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
+----------+-----+--------------------+
only showing top 5 rows

Area under ROC on test data = 0.7184596274839561
Execution time = 2.27855349


import org.apache.spark.ml.classification.{BinaryLogisticRegressionSummary, LogisticRegression}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_ca6518d9a9ec
t1: Long = 113908202201116
lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_ca6518d9a9ec, numClasses = 2, numFeatures = 31
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]
duration: Double = 2.27855349
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_0e25c6358e47
roc1: Double = 0.7184596274839561


### 9.5.2 Decision Tree Model

In [382]:
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
//Build the logistic Regression Model
val dt = new DecisionTreeClassifier().setFeaturesCol("features").setLabelCol("label")

val t1 = System.nanoTime
//Train the Decision Tree Model on trainSet
val dtModel = dt.fit(trainSet)

//Predict on testSet
val predictions = dtModel.transform(testSet)

val duration = (System.nanoTime - t1) / 1e9d

predictions.select("prediction", "label", "features").show(5)


//Judge Model's Performance on test data
val evaluator = new BinaryClassificationEvaluator().setMetricName("areaUnderROC").setLabelCol("label")//.setRawPredictionCol("prediction")
val roc2 = evaluator.evaluate(predictions)
println(s"Area under ROC on test data = $roc2")
println(s"Execution time = $duration")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    1|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
+----------+-----+--------------------+
only showing top 5 rows

Area under ROC on test data = 0.3155095312340184
Execution time = 2.22921509


import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_f32e71053a6d
t1: Long = 113911597600471
dtModel: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_f32e71053a6d) of depth 5 with 35 nodes
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]
duration: Double = 2.22921509
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_9702326bc0cb
roc2: Double = 0.3155095312340184


### 9.5.3 Random Forest Model

In [383]:
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

// Build a Random Forest model.
val rf = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features")//.setNumTrees(500)

val t = System.nanoTime
//Train the Random Forest Model on trainSet
val rfModel = rf.fit(trainSet)

//Predict on testSet
val predictions = rfModel.transform(testSet)

val duration = (System.nanoTime - t) / 1e9d
predictions.select("prediction", "label", "features").show(5)

//Judge Model's Performance on test data
val evaluator = new BinaryClassificationEvaluator().setMetricName("areaUnderROC").setLabelCol("label")//.setRawPredictionCol("prediction")
val roc3 = evaluator.evaluate(predictions)
println(s"Area under ROC on test data = $roc3")
println(s"Execution time = $duration")



+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    1|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
+----------+-----+--------------------+
only showing top 5 rows

Area under ROC on test data = 0.7762402340420517
Execution time = 2.55059438


import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_96e11917e320
t: Long = 113914910423732
rfModel: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_96e11917e320) with 20 trees
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]
duration: Double = 2.55059438
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_2e9058c4dde3
roc3: Double = 0.7762402340420517


## 9.6 Models Comparison

In [384]:
println(s"Logistic Regression: Area under ROC on test data = $roc1")
println(s"Decision Tree: Area under ROC on test data = $roc2")
println(s"Random Forest: Area under ROC on test data = $roc3")

Logistic Regression: Area under ROC on test data = 0.7184596274839561
Decision Tree: Area under ROC on test data = 0.3155095312340184
Random Forest: Area under ROC on test data = 0.7762402340420517


As we can see, the area under ROC of three models are: 
Random Forest > Logistic Regression > Decision Tree.
It is more suitable to use Randon Forest Model to predict the default payment.
However, the time consumption of three models are:
Logistic Regression > Random Forest > Decision Tree,
 Which shows that the Decision tree has the shortest time consumption. 

## 9.7 Improve the Prediction Model

In [385]:
//Adopting parameter tuning to improve Random Forest
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.CrossValidator
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val paramGrid = new ParamGridBuilder().addGrid(rf.numTrees, Array(5, 20, 35)).
                    addGrid(rf.maxDepth, Array(10, 15, 20)).build

val cv = new CrossValidator().setEstimator(rf).setEvaluator(evaluator).
             setEstimatorParamMaps(paramGrid).setNumFolds(3)

val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

//Judge Model's Performance on test data
val roc4 = evaluator.evaluate(predictions)
println(s"Area under ROC on test data = $roc4")


+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    1|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
|       0.0|    0|[-1.2137738805034...|
+----------+-----+--------------------+
only showing top 5 rows

Area under ROC on test data = 0.7851225350709162


import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.CrossValidator
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfc_96e11917e320-maxDepth: 10,
	rfc_96e11917e320-numTrees: 5
}, {
	rfc_96e11917e320-maxDepth: 10,
	rfc_96e11917e320-numTrees: 20
}, {
	rfc_96e11917e320-maxDepth: 10,
	rfc_96e11917e320-numTrees: 35
}, {
	rfc_96e11917e320-maxDepth: 15,
	rfc_96e11917e320-numTrees: 5
}, {
	rfc_96e11917e320-maxDepth: 15,
	rfc_96e11917e320-numTrees: 20
}, {
	rfc_96e11917e320-maxDepth: 15,
	rfc_96e11917e320-numTrees: 35
}, {
	rfc_96e11917e320-maxDepth: 20,
	rfc_96e11917e320-numTrees: 5
}, {
	rfc_96e11917e320-maxDepth: 20,
	rfc_96e11917e320-numTrees: 20
}, {
	rfc_96e11917e320-maxDept...

In order to improve the accuracy, we could delete the correlated rows of the dataset, and clean the not important features. We could also optimize parameters and try other models (SVM, ELM).